In [1]:
!conda uninstall tensorflow -y

Solving environment: failed

PackagesNotFoundError: The following packages are missing from the target environment:
  - tensorflow




In [2]:
!conda install tensorflow -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /Users/orentapiero/miniforge3/envs/tensorenv

  added / updated specs:
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    abseil-cpp-20210324.2      |       hbdafb3b_0         945 KB  conda-forge
    gast-0.3.3                 |             py_0          12 KB  conda-forge
    giflib-5.2.1               |       h27ca646_2          72 KB  conda-forge
    grpc-cpp-1.37.1            |       h538f867_3         2.7 MB  conda-forge
    grpcio-1.37.1              |   py38h69ee544_0         1.8 MB  conda-forge
    h5py-2.10.0                |nompi_py38hb525b2d_106         965 KB  conda-forge
    hdf5-1.10.6              

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import plotly.graph_objects as go
import json
import requests
import datetime
import iso8601

from datetime import datetime
from tqdm import tqdm

sns.set()
plt.rcParams["figure.figsize"] = (18,10)


In [4]:
eth = yf.Ticker("ETH-USD")
ETH = eth.history(period="max")[['Open','High','Low','Close','Volume']]


In [5]:
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

df=ETH.copy()

In [6]:
training_set = df.iloc[:800, 1:2].values
test_set = df.iloc[800:, 1:2].values


In [7]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
# Creating a data structure with 60 time-steps and 1 output
X_train = []
y_train = []

for i in range(60, 800):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#(740, 60, 1)


In [ ]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))#, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)
